In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

max_seq_length = 4096  # 최대 시퀀스 길이를 설정합니다.
dtype = torch.float16 
load_in_4bit = True  # 4bit 양자화 로드 여부를 설정합니다.

model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"  # 모델 이름을 설정합니다.

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=dtype
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    torch_dtype=dtype
)

model.config.max_position_embeddings = max_seq_length

tokenizer = AutoTokenizer.from_pretrained(model_name)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
from peft import get_peft_model, LoraConfig

# LoRA 설정을 정의합니다.
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    use_rslora=False
)

# 모델을 PEFT 방식으로 변환합니다.
model = get_peft_model(model, lora_config)

# gradient checkpointing을 모델의 config에 직접 설정합니다.
model.config.gradient_checkpointing = True


In [48]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


# "teddylee777/QA-Dataset-mini" 데이터셋을 불러옵니다. 훈련 데이터만 사용합니다.
dataset = load_dataset("teddylee777/QA-Dataset-mini", split="train")

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

# 패딩 토큰을 eos_token으로 설정합니다.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [25]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    eval_dataset=dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=100,  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Anaconda3\envs\kollm\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_num_proc. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
c:\Anaconda3\envs\kollm\Lib\site-packages\trl\trainer\sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the valu

Map (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [26]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA GeForce RTX 3070 Laptop GPU. Max memory = 8.0 GB.
5.508 GB of memory reserved.


In [27]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

  0%|          | 0/100 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 13.3672, 'grad_norm': 44.030723571777344, 'learning_rate': 4e-05, 'epoch': 0.5}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_runtime': 56.5091, 'eval_samples_per_second': 0.283, 'eval_steps_per_second': 0.035, 'epoch': 0.5}
{'loss': 13.5156, 'grad_norm': 42.509883880615234, 'learning_rate': 8e-05, 'epoch': 1.0}


...

{'eval_runtime': 25.3264, 'eval_samples_per_second': 0.632, 'eval_steps_per_second': 0.079, 'epoch': 49.0}
{'loss': 0.0822, 'grad_norm': 2.42232346534729, 'learning_rate': 5.467426590739511e-08, 'epoch': 49.5}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_runtime': 25.3434, 'eval_samples_per_second': 0.631, 'eval_steps_per_second': 0.079, 'epoch': 49.5}
{'loss': 0.0764, 'grad_norm': 2.4027607440948486, 'learning_rate': 0.0, 'epoch': 50.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_runtime': 25.3184, 'eval_samples_per_second': 0.632, 'eval_steps_per_second': 0.079, 'epoch': 50.0}
{'train_runtime': 12699.5028, 'train_samples_per_second': 0.063, 'train_steps_per_second': 0.008, 'train_loss': 3.4925225830078124, 'epoch': 50.0}


In [28]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

12699.5028 seconds used for training.
211.66 minutes used for training.
Peak reserved memory = 11.5 GB.
Peak reserved memory for training = 5.992 GB.
Peak reserved memory % of max memory = 143.75 %.
Peak reserved memory for training % of max memory = 74.9 %.


# 저장

In [46]:
# 모델과 토크나이저를 저장할 디렉토리 경로
save_directory = "finetuned_model"

# 모델과 토크나이저를 저장합니다.
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('finetuned_model_merge\\tokenizer_config.json',
 'finetuned_model_merge\\special_tokens_map.json',
 'finetuned_model_merge\\tokenizer.json')

In [47]:
# Hub 에 업로드
# Hugging Face Hub에 모델과 토크나이저를 업로드
model.push_to_hub("Suchae/test-finetuned", use_auth_token="")
tokenizer.push_to_hub("Suchae/test-finetuned", use_auth_token="")

c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\utils\hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Anaconda3\envs\kollm\Lib\site-packages\transformers\utils\hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

c:\Anaconda3\envs\kollm\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\정수채\.cache\huggingface\hub\models--Suchae--test-finetuned-merge. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/Suchae/test-finetuned-merge/commit/0d56d02984ee2ee4081c196813ebef7386d14bc6', commit_message='Upload tokenizer', commit_description='', oid='0d56d02984ee2ee4081c196813ebef7386d14bc6', pr_url=None, pr_revision=None, pr_num=None)